# Applying fit-and-count to mitigate background

Sometimes the sideband subtraction technique might not be suitable, for example, if peaking backgrounds are present in the sample of interest. In such cases, a fit-and-count approach must be employed instead, making use of the knowledge of the components in the distribution to model the signal and each background.

In this tutorial we'll use the `RooFit` implementation of the fitting.

As before, we start with a fairly standard set of imports:

In [1]:
import matplotlib.pyplot as plt
import mplhep as hep
import numpy as np
import ROOT as R
from triggercalib import HltEff
from triggercalib.utils.helpers import tgraph_to_np

hep.style.use("LHCb2")
R.EnableImplicitMT(8)

Welcome to JupyROOT 6.30/04


Considering again the example of the `Hlt1TrackMVA` TOS efficiency in $B^+\to J/\psi\left(\mu\mu\right)K^+$ 2024 MC simulation from the previous tutorial, we will use the $J/\psi K^+$ invariant mass to separate signal and background. We define an observable and PDF, in this case using RooWorkspace (but in principle this can be an existing observable/PDF):

In [2]:
model_description = {
    "signal" : { # Name of component 
        "model" : "CrystalBall", # Name of PDF to be used
        "params" : { # Parameters to be used
            "mu" : [5279.4, 5275, 5285],
            "sigma" : [8.75, 6, 12],
            "alphal" : [1.4, 1.4, 1.4],
            "nl" : [7, 7, 7],
            "alphar" : [1.4, 1.4, 1.4],
            "nr" : [7, 7, 7],
        },
        "yield" : [10000, 0, 1000000] # Associated yield
    },
    "combinatorial" : {
        "model": "Exponential",
        "params" : {
            "exponent" : [-0.001, -0.01, -0.0001]
        },
        "yield": [1000, 0, 1000000]
    }
}

def build_roofit_pdf(ws, model_description):
    model_build_string = "SUM::pdf("
    for component_name, component_description in model_description.items():
        component_build_string = f"{component_description['model']}::{component_name}_pdf(B_DTF_Jpsi_MASS"
        for param_name, (param_val, param_min, param_max) in component_description['params'].items():
            component_build_string += f", {component_name}_{param_name}[{param_val}, {param_min}, {param_max}]"
        component_build_string += ")"
        print(f"Constructing '{component_name}_pdf' using description '{component_build_string}'")
        ws.factory(component_build_string)
        if "yield" in component_description:
            yield_val, yield_min, yield_max = component_description["yield"]
            model_build_string += f", {component_name}_yield[{yield_val}, {yield_min}, {yield_max}]*{component_name}_pdf"
    model_build_string += ")"
    
    print(f"Constructing 'pdf' using description '{model_build_string}'")
    pdf = ws.factory(model_build_string)
    return ws, pdf

ws = R.RooWorkspace("ws")
observable = ws.factory("B_DTF_Jpsi_MASS[5200, 5500]")
ws, pdf = build_roofit_pdf(ws, model_description)

Constructing 'signal_pdf' using description 'CrystalBall::signal_pdf(B_DTF_Jpsi_MASS, signal_mu[5279.4, 5275, 5285], signal_sigma[8.75, 6, 12], signal_alphal[1.4, 1.4, 1.4], signal_nl[7, 7, 7], signal_alphar[1.4, 1.4, 1.4], signal_nr[7, 7, 7])'
Constructing 'combinatorial_pdf' using description 'Exponential::combinatorial_pdf(B_DTF_Jpsi_MASS, combinatorial_exponent[-0.001, -0.01, -0.0001])'
Constructing 'pdf' using description 'SUM::pdf(, signal_yield[10000, 0, 1000000]*signal_pdf, combinatorial_yield[1000, 0, 1000000]*combinatorial_pdf)'


Now that we have an observable and a PDF, we can pass these to `HltEff` in place of our sideband from the previous tutorial, opting for a more coarse binning scheme to reduce the number of fits to run:

In [3]:
binning = {
    "B_PT" : {
        "bins" : [
            2e3, 7e3, 15e3, 25e3
        ]
    }
}

hlt_eff = HltEff(
    "simple_example",
    "root://eoslhcb.cern.ch//eos/lhcb/wg/rta/WP4/TriggerCalib/Bu2JpsiK_Jpsi2MuMu_block1_ntuple.root:Tuple/DecayTree",
    tos="Hlt1TrackMVA",
    tis=["Hlt1TrackMVA", "Hlt1TwoTrackMVA"],
    particle="B", 
    binning=binning,
    observable=observable,
    pdf=pdf,
    fit_kwargs={
        "NumCPU" : 8,
    },
    output_path="fit-and-count/",
    plots=True,
    trig_effs=False, # Avoids running fit for 'trig' category (only interested in TOS and TISTOS here)
)

Fit performed with RooFit from ROOT 6.30/04

Initial parameters:
combinatorial_exponent: -0.001 +/- 0.0
combinatorial_yield: 1000.0 +/- 0.0
signal_alphal: 1.4 +/- 0.0
signal_alphar: 1.4 +/- 0.0
signal_mu: 5279.4 +/- 0.0
signal_nl: 7.0 +/- 0.0
signal_nr: 7.0 +/- 0.0
signal_sigma: 8.75 +/- 0.0
signal_yield: 10000.0 +/- 0.0

Final parameters:
combinatorial_exponent: -0.0019479496497944945 +/- 0.0001398536284415126
combinatorial_yield: 8939.730030214443 +/- 118.95428179855662
signal_alphal: 1.4 +/- 0.0
signal_alphar: 1.4 +/- 0.0
signal_mu: 5279.58383661268 +/- 0.05657418232794953
signal_nl: 7.0 +/- 0.0
signal_nr: 7.0 +/- 0.0
signal_sigma: 8.498370952601476 +/- 0.05159964918657067
signal_yield: 34447.09229525895 +/- 199.14301459686612

Covariance quality: 3
Fit status: 0
Minimum value: -224003.3205400036

Fit performed with RooFit from ROOT 6.30/04

Initial parameters:
combinatorial_exponent: -0.001 +/- 0.0
combinatorial_yield: 1000.0 +/- 0.0
signal_alphal: 1.4 +/- 0.0
signal_alphar: 1.4 +/

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -195262.8459 Edm =       23300.54015 NCalls =     21
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : -195262.8459
  Edm           : 23300.54015
  Internal parameters:	[     0.9582415885     -1.507540228    -0.1202898824   -0.08343008661     -1.370461484]	
  Internal gradient  :	[     -181.2837747     -188709.0707     -232.4906351     -777.3608554     -262848.1493]	
  Internal covariance matrix:
[[   0.0032699471              0              0              0              0]
 [              0   8.094866e-07              0              0              0]
 [              0              0   0.0002559031              0              0]
 [              0              0              0  0.00051101983              0]
 [              0              0              0              0  9.2554826e-07]]]
Info in <Minuit2>: VariableMetricBuild

Excellent, our fits have run and we need now only to plot the efficiency, which we can do exactly as in the first tutorial. To check the individual fits, we can find the fit results and plots of each mass fit in `fit-and-count/`.

In [4]:
hist = hlt_eff.get_eff("tos_efficiency_B_PT")
midpoints, values, xerrors, yerrors = tgraph_to_np(hist, xscale=1e-3)
xmin = midpoints[0] - xerrors[0][0]
xmax = midpoints[-1] + xerrors[1][-1]

# Plot the efficiency #
plt.figure(figsize=(12,10))
plt.gca()

plt.plot((xmin, xmax), (1,1), color='k', ls='dashed', lw=2)
plt.errorbar(
    x=midpoints, y=values, xerr=xerrors, yerr=yerrors,
    color='r', elinewidth=2, ls="none", marker='.', markersize=8, 
)

plt.xlim(xmin, xmax)
plt.xlabel(r"Transverse momentum, $p_T\left(B^+\right)$ / $\mathrm{GeV}c^{-1}$")

plt.ylim(0, 1.1)
plt.ylabel(r"TOS efficiency, $\varepsilon_\mathrm{TOS}$")

hep.lhcb.label(loc=0, rlabel=r"$B^+\to J/\psi\left(\mu\mu\right)K^+$ 2024 MC")

plt.savefig("tos_efficiencies.pdf")
plt.show()